# Stability

In [ ]:
# %load_ext autoreload
# %autoreload 2

import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

%matplotlib inline

### Config

In [2]:
# Experimental setup arguments

class args:

    alpha = 0.2

    delta = 0.1

    epsilon = 0.01

    tau = 0.1

    lambda_max = 1.0

    L = 5.0

    T = 5  # number of iterations

### Model

In [3]:
from data_prep import load_data

path_to_csv_file = './data/cs-training.csv'
X_all, Y_all, data = load_data(path_to_csv_file)

n = X_all.shape[0]
d = X_all.shape[1] - 1

print('n=',n)
print('d=',d)

size = n // 3
X_train, X_cv, Y_train, Y_cv = train_test_split(X_all, Y_all, train_size=size, random_state=42)
del X_all, Y_all

n= 18357
d= 10


In [4]:
# strategic feature indices
strat_features = np.array([1, 6, 8]) - 1 # for later

print('Strategic Features: \n')
for i, feature in enumerate(strat_features):
    print(i, data.columns[feature + 1])

# # zero out non-strategic features
# assert model.coef_.shape == (1, d+1) 
# strat_coef = np.zeros((1, d+1))
# strat_coef[0, strat_features] = model.coef_[0, strat_features]

Strategic Features: 

0 RevolvingUtilizationOfUnsecuredLines
1 NumberOfOpenCreditLinesAndLoans
2 NumberRealEstateLoansOrLines


In [5]:
def train_logistic_regression(X, Y, **kwargs):
    # fit_intercept=False since X already has bias term
    model = LogisticRegression(fit_intercept=False, **kwargs)  # intercept pre-built in X
    model.fit(X, Y)
    assert model.classes_[0] == 0 and model.classes_[1] == 1
    return model

In [6]:
model = train_logistic_regression(X_train, Y_train)
# example_thresh = 0.5
# Y_proba = model.predict_proba(X_train)[:,1]
# Y_pred = Y_proba > example_thresh
print(model.coef_)

[[ 0.01162508 -0.42431799  2.07627747  0.00436746 -0.27894954  0.08004076
   2.55829725  0.04811393  1.45300537  0.09495302 -0.41032303]]


### Set up



In [7]:
from typing import Union

import numpy as np
from sklearn.linear_model import LogisticRegression

import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression



def type_II_error(Y, Y_proba, threshold):
    """Y=actual, Y_proba=predicted probability, threshold=threshold"""
    return np.mean((Y == 1) * (Y_proba < 1 - threshold))


def piecewise_fn(Y_proba, thresh):
    # clipping assumes loss btwn 0 and 1

    # lower_bound = (1 - thresh) - 1 / args.L
    # upper_bound = 1 - thresh
    # y = np.where(Y_proba < lower_bound, 1, 
    #              np.where(Y_proba > upper_bound, 0, 
    #                       1 - args.L * (Y_proba - (1 - thresh) + 1 / args.L)))

    # Optimization 1: use np.clip
    # y = np.clip(1 - args.L * (Y_proba - (1 - thresh) + 1 / args.L), 0, 1)

    # Optimization 2: compute less
    return np.clip(args.L * (1 - thresh - Y_proba), 0, 1).reshape(-1, 1)


def piecewise_loss(Y, Y_proba, thresh):
    return (Y == 1) * piecewise_fn(Y_proba, thresh)


def surrogate_loss(Y, Y_proba, thresh):
    """l_{\tilde}"""
    return piecewise_loss(Y, Y_proba, thresh) + args.tau * (args.lambda_max - thresh)

In [8]:
def get_next_thresh(Y, Y_proba, init_range=None, tol=1e-5, max_iter=30):
    def f(thresh):
        return surrogate_loss(Y, Y_proba, thresh).mean() - args.alpha

    # Choose two initial guesses.
    if init_range is None:
        t0, t1 = 0.0, 1.0
    else:
        t0 = max(0.0, init_range[0])
        t1 = min(1.0, init_range[1])
    
    f0, f1 = f(t0), f(t1)
    
    # If either guess is already close enough, return it.
    if abs(f0) < tol:
        return t0
    if abs(f1) < tol:
        return t1

    for _ in range(max_iter):
        # Prevent division by zero if f1 == f0.
        if f1 - f0 == 0:
            break
        
        # Secant update.
        t_new = t1 - f1 * (t1 - t0) / (f1 - f0)
        # Ensure t_new stays within [0, 1].
        t_new = np.clip(t_new, 0.0, 1.0)
        
        f_new = f(t_new)
        if abs(f_new) < tol:
            return t_new
        
        # Update our guesses.
        t0, f0 = t1, f1
        t1, f1 = t_new, f_new

    return t1


In [9]:
def get_new_features(X, model, thresh):
    # toy "optimization", either stay w/ current features or move by epsilon
    # to new features, depending on which has higher utility cost

    s = np.abs(model.coef_).argmax()
    onehot = np.zeros((1, len(model.coef_[0])))
    onehot[0, s] = 1
    sign = np.sign(model.coef_[0, s])

    # utility of not moving
    stay_utility_cost = np.squeeze(1 - piecewise_fn(model.predict_proba(X)[:,1], thresh))

    # utility of moving by epsilon
    X_move = X + sign * args.epsilon * onehot

    move_utility = np.squeeze(1 - piecewise_fn(model.predict_proba(X_move)[:,1], thresh))
    move_cost = (1 / (2 * args.epsilon / args.L) * np.square(X_move - X)).sum(axis=1)
    move_utility_cost = move_utility - move_cost

    mask = move_utility_cost > stay_utility_cost

    result = np.where(mask[:, np.newaxis], X_move, X)
    return result


In [10]:
from tqdm import tqdm

In [11]:
def run_iters(X, Y, verbose=False):

    losses = []
    threshes = []

    thresh = args.lambda_max
    thresh_diff = args.lambda_max

    iters = tqdm(range(args.T)) if verbose else range(args.T)

    for _ in tqdm(range(args.T)):

        # Deploy threshold
        X_i = get_new_features(X, model, thresh)
        Y_proba = model.predict_proba(X_i)[:,1]

        # Calculate loss
        loss_i = piecewise_loss(Y, Y_proba, thresh).mean()

        # Update to new threshold, expect thresh difference to continue decreasing
        thresh_new = get_next_thresh(Y,
                                     Y_proba,
                                     init_range=(thresh - thresh_diff, thresh + thresh_diff),)
        thresh_diff = abs(thresh - thresh_new)
        thresh = thresh_new

        losses.append(loss_i)
        threshes.append(thresh)

    return thresh, losses, threshes

In [12]:
import numpy as np

def simulate_thresh_sensitivity(X_cv, Y_cv, num_trials=2):
    # Run on the full dataset to get the original threshold.
    orig_thresh, _, _ = run_iters(X_cv, Y_cv)

    thresh_list = []
    n = len(X_cv)
    
    # Ensure we don't exceed the number of available points.
    if num_trials > n:
        raise ValueError("num_trials cannot exceed the number of data points in X_cv.")

    # Get a random permutation of indices and select the first num_trials indices
    indices = np.random.permutation(n)[:num_trials]

    for idx in tqdm(indices):
        # Remove the selected data point from X_cv and Y_cv
        X_new = np.delete(X_cv, idx, axis=0)
        Y_new = np.delete(Y_cv, idx, axis=0)
        
        # Run run_iters on the modified dataset
        new_thresh, _, _ = run_iters(X_new, Y_new)
        thresh_list.append(new_thresh)
    
    # Convert to a NumPy array for vectorized operations
    thresh_array = np.array(thresh_list)
    
    # Calculate the maximum absolute difference from the original threshold
    max_diff = np.max(np.abs(thresh_array - orig_thresh))
    
    return max_diff


In [13]:
max_diff = simulate_thresh_sensitivity(X_cv, Y_cv, num_trials=200)
max_diff

  0%|          | 0/5 [00:00<?, ?it/s]

100%|██████████| 200/200 [51:06<00:00, 15.33s/it]


np.float64(3.620011382010624e-05)